In [1]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

```
SELECT 
doc.company_id, 
sli.document_id,
sli.external_id,
doc.doc_date::DATE, 
sli.item_number, 
sli.item_description,
sli.is_component,
sli.is_package,
sli.is_noninventory,
sli.is_dropship,
sli.unit_cost,
sli.unit_price,
sli.extended_cost,
sli.extended_price,
sli.markdown_amount,
sli.discount_amount,
sli.quantity,
sli.price_level
FROM ingress_saleslineitem sli
join reporting_latestdocexclusionsview as doc on sli.document_id = doc.id
join company_company as co on sli.company_id = co.id
where doc.document_class = 3
and not co.is_archived
and not (is_noninventory and not is_package)
and not true_void
ORDER BY company_id ASC
```


In [2]:
df = pd.read_csv('data.csv')
print(df.shape)
df.head()

(1690045, 18)


,company_id,document_id,external_id,doc_date,item_number,item_description,is_component,is_package,is_noninventory,is_dropship,unit_cost,unit_price,extended_cost,extended_price,markdown_amount,discount_amount,quantity,price_level
0,2b4a12c8-80be-4654-b58b-1d3d98226cd4,4c6a9169-31d6-4ee5-afce-c6fec611db90,INVOICE|INV-0695568|16384|0,2021-11-16,07202000,"Corning U-Shaped Cell Culture Flasks, Capacity...",False,False,False,True,178.470,178.470,1070.820,1070.820,0.000,0.000,6.000,RETAIL
1,2b4a12c8-80be-4654-b58b-1d3d98226cd4,aad17e8a-643b-4b53-8088-e0fd10382b5e,INVOICE|INV-0448217|16384|0,2018-08-07,RAINABV-30389225,Rainin Instruments LLC RT-LTS-A-10ÂµL-/F-960/1...,False,False,False,True,64.870,69.740,259.480,278.960,0.000,0.000,4.000,RETAIL
2,2b4a12c8-80be-4654-b58b-1d3d98226cd4,09d9ea24-94ce-4a09-9c64-4789dfeaf912,INVOICE|INV-0905303|32768|0,2023-08-29,SIAL-D-073-1ML,Sigma-Aldrich 5alpha-Dihydrotestosterone (Dht)...,False,False,False,True,109.600,129.460,109.600,129.460,0.000,0.000,1.000,RETAIL
3,2b4a12c8-80be-4654-b58b-1d3d98226cd4,2b9ee1db-82d0-4840-9469-c706b98eaf63,INVOICE|INV-0467165|16384|0,2018-11-15,0100-1855,"Agilent Rotor seal, Vespel, for p/n 0101-0921 ...",False,False,False,False,84.600,102.930,338.400,411.720,0.000,0.000,4.000,RETAIL
4,2b4a12c8-80be-4654-b58b-1d3d98226cd4,2b9ee1db-82d0-4840-9469-c706b98eaf63,INVOICE|INV-0467165|32768|0,2018-11-15,0101-1416,"Agilent Rotor seal, 3 grooves, max 600 bar for...",False,False,False,False,295.200,359.160,1476.000,1795.800,0.000,0.000,5.000,RETAIL


In [3]:
df.groupby(['company_id']).agg({'external_id':'size','is_component':'sum','is_package':'sum'})

,external_id,is_component,is_package
company_id,,,
2b4a12c8-80be-4654-b58b-1d3d98226cd4,799950,697,162
94c89416-8bff-4d2d-96c9-1f3f1ff22a97,187580,23124,7253
acfcb7b7-2931-49b4-8829-2833124631b5,229368,607,187
ad8e56d3-831d-4fc4-8747-1fecb961a9d8,6203,0,0
b2cbafb2-6bb1-4bd0-a284-b4bf04885de6,44072,142,72
f3dc8773-d687-47b9-887d-6442c32d155c,422872,21074,18766


In [4]:
def clean_df(df):
    
    df = df.copy()
    
    orig_cols = list(df.columns)
    
    df['doc_date'] = pd.to_datetime(df['doc_date'])
    
    df['line_number'] = df['external_id'].map(lambda i: int(i.split('|')[2]) if len(i.split('|')) > 1 and i.split('|')[2].isdigit() else np.nan )
    
    df['subline_number'] = df['external_id'].map(lambda i: int(i.split('|')[3]) if len(i.split('|')) > 3 and i.split('|')[3].isdigit() else np.nan )
    
    df = df.sort_values(['company_id','document_id','line_number'])
    
    date_max = df.doc_date.max()

    df = df[(df.quantity > 0)&(df.doc_date>=(date_max-relativedelta(years=2)))]
    
    df['doc_date'] = df['doc_date'].dt.date
    
    
#     df_standalone = df[~(df.is_package|df.is_component)]
    
#     print(df_standalone.shape)
    
    df_packages = df[(df.is_package|df.is_component)]
    
    df_packages = df_packages.sort_values(['company_id','document_id','line_number','subline_number','is_package'],ascending=[True,True,True,True,False]).reset_index(drop=True)
    
    print(df_packages.shape)
    

    
    new_columns = []
    
    curr_package_info = {}
    
    metrics_to_copy = ['unit_cost','unit_price','extended_cost','extended_price','markdown_amount','discount_amount']
    
    df_packages.loc[df_packages['is_package'],'group_id'] = df_packages['external_id']
    
    df_packages['group_id'] = df_packages['group_id'].ffill()
    
    df_packages.loc[df_packages['is_package'],'group_doc_id'] = df_packages['document_id']
    
    df_packages['group_doc_id'] = df_packages['group_doc_id'].ffill()
    
    for m in metrics_to_copy:
        
        df_packages.loc[df_packages['is_package'],f'{m}_pack'] = df_packages[m]
        
        df_packages.loc[df_packages['is_package'],f'{m}_temp'] = df_packages[m]
        
        df_packages[f'{m}_pack'] = df_packages[f'{m}_pack'].ffill()
        
        df_packages.loc[
            (
                df_packages['is_component']
                &(df_packages['document_id'] == df_packages['group_doc_id'])
                &(df_packages[f'{m}_pack'] == 0)
            ),f'{m}_temp'
        ] = df_packages[m]
        
        df_packages.loc[
            (
                df_packages['is_component']
                &(df_packages['document_id'] == df_packages['group_doc_id'])
                &(df_packages[f'{m}_pack'] != 0)
            ),f'{m}_temp'
        ] = 0
        
    df_packages_grouped = df_packages.groupby([
        'company_id', 
        'document_id', 
        'group_id', 
    ])[[f'{m}_temp' for m in metrics_to_copy]].sum().reset_index().rename(columns={'group_id':'external_id'})
    
    df = df.merge(df_packages_grouped,how='left',on=['company_id', 'document_id', 'external_id'])
    
    for m in metrics_to_copy:
        df.loc[(~df[f'{m}_temp'].isnull())&(df[f'{m}_temp']!=df[m]),m] = df[f'{m}_temp']
        
    df = df[~df.is_component].reset_index(drop=True)[orig_cols]
    
    df.loc[(df.extended_cost==0)&df.unit_cost!=0,'extended_cost'] = df.unit_cost*df.quantity
           
    df.loc[df.extended_cost!=0,'unit_cost'] = df.extended_cost/df.quantity
    
    df.loc[(df.extended_price==0)&df.unit_price!=0,'extended_price'] = df.unit_price*df.quantity
           
    df.loc[df.extended_price!=0,'unit_price'] = df.extended_price/df.quantity
    
    df.loc[df.extended_price!=0,'unit_price_with_markdown'] = (df.extended_price+df.markdown_amount)/df.quantity
    
    df['unit_discount'] = df.discount_amount/df.quantity
    
    df['unit_margin'] = df['unit_price'] - df['unit_discount']-df['unit_cost']
    
    df.price_level = df.price_level.fillna('')
    
    return df

df_clean = clean_df(df)

df_clean.head(50)

(23601, 20)


,company_id,document_id,external_id,doc_date,item_number,item_description,is_component,is_package,is_noninventory,is_dropship,unit_cost,unit_price,extended_cost,extended_price,markdown_amount,discount_amount,quantity,price_level,unit_price_with_markdown,unit_discount,unit_margin
0,2b4a12c8-80be-4654-b58b-1d3d98226cd4,000003d4-73f8-49a0-83b8-971b414f5f46,INVOICE|INV-0735412|49152|0,2022-04-08,5185-5959,Agilent Tuning Solution for ICP-MS 7500cs - 58...,False,False,False,True,231.800,277.400,231.800,277.400,0.000,0.000,1.000,RETAIL,277.400,0.000,45.600
1,2b4a12c8-80be-4654-b58b-1d3d98226cd4,0000ad41-75bd-476a-a23e-421fc8ce8475,INVOICE|INV-0871920|16384|0,2023-05-31,695675-742,Agilent Technologies Poroshell 120 Aq-C18 3.0x...,False,False,False,False,630.300,792.650,630.300,792.650,0.000,0.000,1.000,RETAIL,792.650,0.000,162.350
2,2b4a12c8-80be-4654-b58b-1d3d98226cd4,0000c343-40cd-494d-9afd-0e3f9869bf1d,INVOICE|INV-0853862|16384|0,2023-04-11,CSIG-7727S,Cell Signaling Biotinylated Protein Ladder Det...,False,False,False,True,72.900,79.790,145.800,159.580,0.000,0.000,2.000,RETAIL,79.790,0.000,6.890
3,2b4a12c8-80be-4654-b58b-1d3d98226cd4,0000dd6f-ffd9-4bf9-b37a-b5e266fcead4,INVOICE|INV-0710796|32768|0,2022-01-13,SIALGSK-A4599-100G,Sigma Aldrich L-ARGININE HYDROCHLORIDE 100G-...,False,False,False,True,112.360,117.040,112.360,117.040,0.000,0.000,1.000,RETAIL,117.040,0.000,4.680
4,2b4a12c8-80be-4654-b58b-1d3d98226cd4,0000e58c-39ad-4da9-8fc9-4a7dfd281247,INVOICE|INV-0882834|16384|0,2023-06-28,CHMGLS-CG-3080-01,"Chemglass Life Sciences Centric Syringe, 1 mL ...",False,False,False,True,17.930,21.830,35.860,43.660,0.000,0.000,2.000,RETAIL,21.830,0.000,3.900
5,2b4a12c8-80be-4654-b58b-1d3d98226cd4,0000f5aa-039d-4e4c-9418-4b9c0c66fae8,INVOICE|INV-0911846|16384|0,2023-09-18,MILL-SLHVR33RS,"Millipore Millex Slhvr33rs Syringe Filter, 0.4...",False,False,False,True,180.120,213.410,540.360,640.230,0.000,0.000,3.000,RETAIL,213.410,0.000,33.290
6,2b4a12c8-80be-4654-b58b-1d3d98226cd4,00010214-9b8e-42bf-aa48-93f8e59b07fc,INVOICE|INV-0834568|16384|0,2023-02-20,SIALGSK-151874-10X1ML,Sigma Aldrich DIMETHYL SULFOXIDE-D6 99.9 ATOM...,False,False,False,True,8.840,9.200,88.400,92.000,0.000,0.000,10.000,RETAIL,9.200,0.000,0.360
7,2b4a12c8-80be-4654-b58b-1d3d98226cd4,00010214-9b8e-42bf-aa48-93f8e59b07fc,INVOICE|INV-0834568|32768|0,2023-02-20,SIALGSK-151823-100G,Sigma Aldrich CHLOROFORM-D 99.8 ATOM % D- SIA...,False,False,False,True,8.970,9.340,53.820,56.040,0.000,0.000,6.000,RETAIL,9.340,0.000,0.370
8,2b4a12c8-80be-4654-b58b-1d3d98226cd4,00010214-9b8e-42bf-aa48-93f8e59b07fc,INVOICE|INV-0834568|49152|0,2023-02-20,SIALGSK-34860-4X2L-R,Sigma Aldrich METHANOL FOR HPLC >99.9%- SIAL...,False,False,False,True,45.240,47.120,45.240,47.120,0.000,0.000,1.000,RETAIL,47.120,0.000,1.880
9,2b4a12c8-80be-4654-b58b-1d3d98226cd4,00010214-9b8e-42bf-aa48-93f8e59b07fc,INVOICE|INV-0834568|65536|0,2023-02-20,SIALGSK-270725-4X2L,Sigma Aldrich ACETONE FOR HPLC >99.9%- SIALG...,False,False,False,True,50.600,52.710,50.600,52.710,0.000,0.000,1.000,RETAIL,52.710,0.000,2.110


In [5]:
def get_differences_per_day(df,threshold_percent=0):
    
    df = df.copy()
    
    for c in ['unit_cost','unit_price','unit_price_with_markdown','markdown_amount','unit_discount','unit_margin']:
        
        df[c] = df[c].astype(int,errors='ignore')
    
        df_c_ts_min = df.groupby(['company_id','item_number','price_level','doc_date'])[c].min().reset_index().sort_values(['company_id','item_number','price_level','doc_date'])
        
        if 'previous_doc_date' not in df.columns:
            
            df_c_ts_min.loc[
                (
                    (df_c_ts_min.company_id==df_c_ts_min.company_id.shift(1))
                    &(df_c_ts_min.item_number==df_c_ts_min.item_number.shift(1))
                    &(df_c_ts_min.price_level==df_c_ts_min.price_level.shift(1))
                ),
                'previous_doc_date'
            ] = df_c_ts_min['doc_date'].shift(1)
        
        df_c_ts_min.loc[
            (
                (df_c_ts_min.company_id==df_c_ts_min.company_id.shift(1))
                &(df_c_ts_min.item_number==df_c_ts_min.item_number.shift(1))
                &(df_c_ts_min.price_level==df_c_ts_min.price_level.shift(1))
            ),
            f'min_previous_{c}'
        ] = df_c_ts_min[c].shift(1)
    
        del df_c_ts_min[c]
        
        df_c_ts_max = df.groupby(['company_id','item_number','price_level','doc_date'])[c].max().reset_index().sort_values(['company_id','item_number','price_level','doc_date'])
        
        df_c_ts_max.loc[
            (
                (df_c_ts_max.company_id==df_c_ts_max.company_id.shift(1))
                &(df_c_ts_max.item_number==df_c_ts_max.item_number.shift(1))
                &(df_c_ts_max.price_level==df_c_ts_max.price_level.shift(1))
            ),
            f'max_previous_{c}'
        ] = df_c_ts_max[c].shift(1)
        
        del df_c_ts_max[c]
        
        df = df.merge(df_c_ts_min,on=['company_id','item_number','price_level','doc_date'],how='left')
        
        df = df.merge(df_c_ts_max,on=['company_id','item_number','price_level','doc_date'],how='left')
        
        df['days_since_last'] = (df['doc_date'] - df['previous_doc_date']).dt.days
        
        df[f'gt_max_previous_{c}'] = df[c] > df[f'max_previous_{c}'] * (1 + threshold_percent)
        
        df[f'lt_min_previous_{c}'] = df[c] < df[f'min_previous_{c}'] * (1 - threshold_percent)
        
        df.loc[(df[f'gt_max_previous_{c}'])|(df[f'lt_min_previous_{c}']),f'day_since_previous_diff_{c}'] = df['days_since_last'] 

    df = df.sort_values(['company_id','item_number','price_level','doc_date']).reset_index(drop=True)
    
    
    return df

df_comparisons = get_differences_per_day(df_clean,threshold_percent=0.15)

df_comparisons

,company_id,document_id,external_id,doc_date,item_number,item_description,is_component,is_package,is_noninventory,is_dropship,unit_cost,unit_price,extended_cost,extended_price,markdown_amount,discount_amount,quantity,price_level,unit_price_with_markdown,unit_discount,unit_margin,previous_doc_date,min_previous_unit_cost,max_previous_unit_cost,days_since_last,gt_max_previous_unit_cost,lt_min_previous_unit_cost,day_since_previous_diff_unit_cost,min_previous_unit_price,max_previous_unit_price,gt_max_previous_unit_price,lt_min_previous_unit_price,day_since_previous_diff_unit_price,min_previous_unit_price_with_markdown,max_previous_unit_price_with_markdown,gt_max_previous_unit_price_with_markdown,lt_min_previous_unit_price_with_markdown,day_since_previous_diff_unit_price_with_markdown,min_previous_markdown_amount,max_previous_markdown_amount,gt_max_previous_markdown_amount,lt_min_previous_markdown_amount,day_since_previous_diff_markdown_amount,min_previous_unit_discount,max_previous_unit_discount,gt_max_previous_unit_discount,lt_min_previous_unit_discount,day_since_previous_diff_unit_discount,min_previous_unit_margin,max_previous_unit_margin,gt_max_previous_unit_margin,lt_min_previous_unit_margin,day_since_previous_diff_unit_margin
0,2b4a12c8-80be-4654-b58b-1d3d98226cd4,5f396f06-1798-4c84-8228-9295dec0690d,INVOICE|INVS-1973|24576|0,2021-11-08,*AGICAP-3672254-G5498B*****,CAPITAL EQUIPMENT PURCHASES,False,False,False,True,3159.000,3285.360,3159.000,3285.360,0.000,0.000,1.000,TIER1,3285.360,0,126.360,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN
1,2b4a12c8-80be-4654-b58b-1d3d98226cd4,5f396f06-1798-4c84-8228-9295dec0690d,INVOICE|INVS-1973|8192|0,2021-11-08,*AGICAP-3672254-G5563AA*****,G5563AA Bravo Liquid Handler Bundle,False,False,False,True,101775.000,105846.000,101775.000,105846.000,0.000,0.000,1.000,TIER1,105846.000,0,4071.000,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN
2,2b4a12c8-80be-4654-b58b-1d3d98226cd4,765e1f2e-2e5a-4bc7-a77c-3fd5f24c9b3a,INVOICE|INVS-5509|40960|0,2023-09-28,*LPS-SQ038357- 933.0064.016,Bioprene tubing 6.4mmbore x 1.6mm wall thickness,False,False,False,True,28.310,29.300,424.650,439.500,0.000,0.000,15.000,TIER1,29.300,0,0.990,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN
3,2b4a12c8-80be-4654-b58b-1d3d98226cd4,765e1f2e-2e5a-4bc7-a77c-3fd5f24c9b3a,INVOICE|INVS-5509|24576|0,2023-09-28,*LPS-SQ038357-913.A064.016,Pumpsil Tubing 6.4mm Bore X 1.6mm Wall Thickness,False,False,False,True,23.370,24.190,350.550,362.850,0.000,0.000,15.000,TIER1,24.190,0,0.820,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN
4,2b4a12c8-80be-4654-b58b-1d3d98226cd4,62f6f387-7a76-4a99-9ec5-66a72e9ea306,INVOICE|INV-0684629|16384|0,2021-10-08,00000001600L,"Agilent Test tubes, polypropylene, 17 mm od, 1...",False,False,False,False,182.310,244.110,364.620,488.220,0.000,0.000,2.000,RETAIL,244.110,0,61.800,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607856,f3dc8773-d687-47b9-887d-6442c32d155c,dfed0da7-9924-4ed5-958d-3edac214c52a,INVOICE|167939|90112|0,2022-05-04,TEA-600749,"TX Grade 4, Module 3",False,False,False,False,21.670,76.000,780.120,2736.000,0.000,0.000,36.000,EXTPRCLVL,76.000,0,54.330,2022-05-03,21.670,21.670,1.000,False,False,NaN,76.000,76.000,False,False,NaN,76.000,76.000,False,False,NaN,0.000,0.000,False,

In [9]:
def get_item_importance(threshold=0.9):

    temp_df = df_clean[df_clean.doc_date>=(datetime.now()-relativedelta(years=2)).date()].groupby(['company_id','item_number'])['extended_price','extended_cost','discount_amount'].sum().reset_index()
    
    temp_df['total_margin'] = temp_df['extended_price'] - temp_df['extended_cost'] - temp_df['discount_amount']
    
    temp_df = temp_df.sort_values(['company_id','total_margin'],ascending=False)
    
    temp_df['total_margin_cumsum'] = temp_df.groupby(['company_id'])['total_margin'].cumsum()
    
    comp_df = temp_df.groupby(['company_id']).total_margin.sum().reset_index().rename(columns={'total_margin':'total_company_margin'})
    
    temp_df = temp_df.merge(comp_df,on='company_id')
    
    temp_df['total_margin_cumsum_pc'] = round(temp_df['total_margin_cumsum']/temp_df['total_company_margin'],2)
    
    temp_df['important_item'] = temp_df['total_margin_cumsum_pc'] <= threshold
    
    return temp_df
    
get_item_importance().groupby(['company_id']).important_item.value_counts()/get_item_importance().groupby(['company_id']).important_item.size()

/var/folders/wm/wmwjydb54bj74n244wgkmg3w0000gn/T/ipykernel_74321/919641797.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp_df = df_clean[df_clean.doc_date>=(datetime.now()-relativedelta(years=2)).date()].groupby(['company_id','item_number'])['extended_price','extended_cost','discount_amount'].sum().reset_index()
/var/folders/wm/wmwjydb54bj74n244wgkmg3w0000gn/T/ipykernel_74321/919641797.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp_df = df_clean[df_clean.doc_date>=(datetime.now()-relativedelta(years=2)).date()].groupby(['company_id','item_number'])['extended_price','extended_cost','discount_amount'].sum().reset_index()


company_id                            important_item
2b4a12c8-80be-4654-b58b-1d3d98226cd4  False            0.856
                                      True             0.144
94c89416-8bff-4d2d-96c9-1f3f1ff22a97  False            0.739
                                      True             0.261
acfcb7b7-2931-49b4-8829-2833124631b5  False            0.825
                                      True             0.175
ad8e56d3-831d-4fc4-8747-1fecb961a9d8  False            0.916
                                      True             0.084
b2cbafb2-6bb1-4bd0-a284-b4bf04885de6  False            0.840
                                      True             0.160
f3dc8773-d687-47b9-887d-6442c32d155c  False            0.877
                                      True             0.123
Name: important_item, dtype: float64

In [ ]:
def get_counts_per_day(df):
    
    df = df.copy()
    
    df_count_lines = df.groupby(['company_id','doc_date']).size().reset_index().rename(columns={0:'total_lines_day'})
    
    df_average_days_since =  df.groupby(['company_id','doc_date'])[['days_since_last']+[c for c in df.columns if 'day_since_previous' in c]].median().reset_index().rename(columns={'days_since_last':'median_days_since_last_change'})
    
    df_group = df.groupby(['company_id','doc_date'])[[c for c in df.columns if 'lt_min' in c or 'gt_max' in c]].sum().reset_index()
    
    df_count_lines = df_count_lines.merge(df_group,on=['company_id','doc_date'],how='outer')
    
    df_count_lines = df_count_lines.merge(df_average_days_since,on=['company_id','doc_date'],how='outer')
    
    df_min_dates = df_count_lines.groupby(['company_id'])['doc_date'].min().reset_index().rename(columns={'doc_date':'first_date'})
    
    df_count_lines = df_count_lines.merge(df_min_dates,on='company_id',how='left')
    
    df_count_lines = df_count_lines[df_count_lines['doc_date'] != df_count_lines['first_date']].reset_index(drop=True)

    del df_count_lines['first_date']
    
    df_count_lines['doc_date'] = pd.to_datetime(df_count_lines['doc_date'])
    
    print(df_count_lines['doc_date'].max() - relativedelta(months=12))
    
    df_count_lines = df_count_lines[df_count_lines['doc_date'].max() - relativedelta(months=12)<df_count_lines['doc_date']].reset_index(drop=True)
    
    return df_count_lines

df_counts = get_counts_per_day(df_comparisons)

df_counts

In [ ]:
def get_metrics(df):
    
    df = df.copy()
    
    df_all_metrics = df.groupby(['company_id'])[[c for c in df.columns if 'lt_min' in c or 'gt_max' in c]].describe(percentiles=[.25, .5, .75]).reset_index()
    
    df_all_metrics.columns = [c[0]+'_'+c[1] if c[1] != '' else c[0] for c in df_all_metrics.columns]
    
    df_add_metrics = df.groupby(['company_id'])[[c for c in df.columns if 'total_lines_day' in c or 'day_since_previous' in c]].median().reset_index().rename(columns={c:'median_'+c for c in df.columns if 'total_lines_day' in c or 'day_since_previous' in c})
    
    df_all_metrics = df_all_metrics.merge(df_add_metrics,on='company_id').round(0)
    
    replacements = {
        'gt_max_previous_':'> Last',
        'lt_min_previous_':'< Last',
        'unit_cost':' Cost',
        'unit_price':' Price',
        'markdown_amount':' Markdown',
        'unit_discount':' Discount',
        'unit_margin':' Margin',
        'mean':' Mean',
        'min':' Min',
        'max':' Max',
        '25%':' 1Q',
        '50%':' Median',
        '75%':' 3Q',
        'max':' Max',
        'median_total_lines_day':'Total Lines per day Median',
        'median_day_since_previous_diff_':'Median days since last change -'
    }
    
    for c in df_all_metrics.columns:
        
        if '_count' in c or 'std' in c:
            
            del df_all_metrics[c]
            
        else:
            
            new_column_name = c
            
            for r in replacements:
                
                new_column_name = new_column_name.replace(r,replacements[r])
                
            new_column_name = new_column_name.replace('_',' ')
                
            df_all_metrics = df_all_metrics.rename(columns={c:new_column_name})
        
    
    return df_all_metrics

final_results_df = get_metrics(df_counts)

final_results_df

In [ ]:
final_results_df.columns